In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm
import pickle
import os
import multiprocessing
from PF_Aso1_0_CPU import *

workdir = os.path.dirname(os.getcwd())
srcdir = os.getcwd()
datadir = workdir + '/data/'
outputdir = workdir + '/output/'

In [2]:
if __name__ == '__main__':

    workdir = os.path.dirname(os.getcwd())
    srcdir = os.getcwd()
    datadir = workdir + '/data/'
    outputdir = workdir + '/output/'

    seed = 7

    obs_series = pd.read_csv(datadir + 'data.csv', delimiter=',')
    obs_series = np.array(obs_series.iloc[:,1:]).T

    T = obs_series.shape[1]
    N = 5000
    Λ_scale = 1.0
    cd_scale = 1.0

    case = 'actual data, seed = ' + str(seed) + ', T = ' + str(T) + ', N = ' + str(N) + ', Λ_scale = ' + str(Λ_scale) + ', cd_scale = ' + str(cd_scale)
    try: 
        casedir = outputdir + case  + '/'
        os.mkdir(casedir)
    except:
        casedir = outputdir + case  + '/'

    np.random.seed(seed)
    start_time = time.time()

    D_0 = obs_series[:,[0]]

    Input_0 = [[D_0, Λ_scale, cd_scale] for i in range(N)]
    pool = multiprocessing.Pool()
    Output_0 = pool.map(init, tqdm(Input_0))
    θ_t_particle = [i[0] for i in Output_0]
    X_t_particle = [i[1] for i in Output_0]
    H_t_particle = [i[2] for i in Output_0]

    with open(casedir + 'θ_0.pkl', 'wb') as f:
        pickle.dump(θ_t_particle, f)
    with open(casedir + 'X_0.pkl', 'wb') as f:
        pickle.dump(X_t_particle, f)
    with open(casedir + 'H_0.pkl', 'wb') as f:
        pickle.dump(H_t_particle, f)
    with open(casedir + 'count_0.pkl', 'wb') as f:
        pickle.dump(list(np.ones(N)), f)
    with open(casedir + 'w_0.pkl', 'wb') as f:
        pickle.dump(list(np.ones(N)/N), f)
    run_time = time.time() - start_time
    print(run_time)    

    del(θ_t_particle)

    for t in tqdm(range(T-1)):
        
        D_t_next = obs_series[:,[t+1]]
        
        Input = [[D_t_next, X_t_particle[i], H_t_particle[i], seed+t+i] for i in range(N)]
        pool = multiprocessing.Pool()
        Output = pool.map(recursive, tqdm(Input))
          
        θ_t_next_particle = [i[0] for i in Output]
        X_t_next_particle = [i[1] for i in Output]
        H_t_next_particle = [i[2] for i in Output]
        ν_t_next_particle = [i[3] for i in Output]    
        
        with open(casedir + 'θ_' + str(t+1) + '.pkl', 'wb') as f:
            pickle.dump(θ_t_next_particle, f)
        with open(casedir + 'X_' + str(t+1) + '.pkl', 'wb') as f:
            pickle.dump(X_t_next_particle, f)
        with open(casedir + 'H_' + str(t+1) + '.pkl', 'wb') as f:
            pickle.dump(H_t_next_particle, f)
        
        w_t_next = ν_t_next_particle/np.sum(ν_t_next_particle)
        try:
            count_all = sp.stats.multinomial.rvs(N, w_t_next)
        except:
            for i in range(w_t_next.shape[0]):
                if w_t_next[i]>(np.sum(w_t_next[:-1]) - 1):
                    w_t_next[i] = w_t_next[i] - (np.sum(w_t_next[:-1]) - 1)
                    break
            count_all = sp.stats.multinomial.rvs(N, w_t_next)
        
        with open(casedir + 'w_' + str(t+1) + '.pkl', 'wb') as f:
            pickle.dump(w_t_next, f)
        with open(casedir + 'count_' + str(t+1) + '.pkl', 'wb') as f:
            pickle.dump(count_all, f)
        
        del(X_t_particle)
        del(H_t_particle)
        del(w_t_next)
        del(D_t_next)
        del(θ_t_next_particle)
        del(ν_t_next_particle)
        
        X_t_particle = []
        H_t_particle = []
        
        for i in range(N):
            if count_all[i] != 0:
                for n in range(count_all[i]):
                    X_t_particle.append(X_t_next_particle[i])
                    H_t_particle.append(H_t_next_particle[i])
        
        del(count_all)            
        del(X_t_next_particle)
        del(H_t_next_particle)
        

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:07<00:00, 657.63it/s]


15.276580095291138


 25%|███████████████████████████████████████▉                                                                                                                       | 1256/5000 [00:00<00:00, 9108.45it/s]Process SpawnPoolWorker-22:
Process SpawnPoolWorker-24:
Exception ignored on calling ctypes callback function: <function ExecutionEngine._raw_object_cache_notify at 0x7fb160cd3550>
Traceback (most recent call last):
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/site-packages/llvmlite/binding/executionengine.py", line 171, in _raw_object_cache_notify
Exception ignored on calling ctypes callback function: <function ExecutionEngine._raw_object_cache_notify at 0x7fd5402c3550>
Traceback (most recent call last):
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/site-packages/llvmlite/binding/executionengine.py", line 171, in _raw_object_cache_notify
Process SpawnPoolWorker-17:
  0%|▌                                                                                                      

Process SpawnPoolWorker-18:
Traceback (most recent call last):
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/Users/haominqin/Dropbox/Mac/Documents/Estimation Projects/44th Version, PF large parallel/src/PF_Aso1_0_CPU.py", line 275, in recursive
    bayes_para_update(Ht[1][0], Ht[1][1], Ht[1][2], Ht[1][3], np.array([[1],[Zt2]]), Zt_next_2)
  File "/Users/haominqin/opt/anaconda3/lib/python3.9/site-packages/numba/core/dispatcher.py", line 420, in _compile_for_args
    return_val = self.compile(tuple(ar

KeyboardInterrupt: 

In [7]:
seed = 7

obs_series = pd.read_csv(datadir + 'data.csv', delimiter=',')
obs_series = np.array(obs_series.iloc[:,1:]).T

T = obs_series.shape[1]
N = 5000
Λ_scale = 1.0
cd_scale = 1.0

case = 'actual data, seed = ' + str(seed) + ', T = ' + str(T) + ', N = ' + str(N) + ', Λ_scale = ' + str(Λ_scale) + ', cd_scale = ' + str(cd_scale)
try: 
    casedir = outputdir + case  + '/'
    os.mkdir(casedir)
except:
    casedir = outputdir + case  + '/'

In [8]:
def recursive(Dt_next, Xt, Ht):
    
    stability_max_iter = 200_000
    
    β_z1, σ2_z1 = draw_para(Ht[0][0], Ht[0][1], Ht[0][2], Ht[0][3])
    b11 = np.sqrt(σ2_z1)
    
    λ_stable = False
    λ_iter = 0
    while λ_stable == False:
        β_z2, σ2_z2 = draw_para(Ht[1][0], Ht[1][1], Ht[1][2], Ht[1][3])
        η = β_z2[0,0] 
        λ = β_z2[1,0] 
        b22 = np.sqrt(σ2_z2)
        λ_iter += 1
        if abs(λ) < 0.99:
            λ_stable = True
        elif λ_iter > stability_max_iter:
            print('λ Unstable')
        else:
            λ_stable = False

    Azo = np.array([[0],[η]])
    Azz = np.array([[1, 1],[0, λ]])
    Bz = np.array([[b11, 0],[0, b22]])
    
    Ass_stable = False
    Ass_iter = 0
    
    while Ass_stable == False:
        β_s1, σ2_s1 = draw_para(Ht[2][0], Ht[2][1], Ht[2][2], Ht[2][3])
        β_s2, σ2_s2 = draw_para(Ht[3][0], Ht[3][1], Ht[3][2], Ht[3][3])
        β_s3, σ2_s3 = draw_para(Ht[4][0], Ht[4][1], Ht[4][2], Ht[4][3])

        J_inv = np.array([[1.0,                0.0,        0.0],\
                          [-β_s2[1,0],         1.0,        0.0],\
                          [-β_s3[1,0], -β_s3[2,0],         1.0]])
        Aso = sp.linalg.solve(J_inv, np.array([[0], β_s2[0], β_s3[0]]))
        Ass = sp.linalg.solve(J_inv, np.array([[β_s1[0,0], β_s1[1,0], β_s1[2,0]],\
                                               [β_s2[2,0], β_s2[3,0], β_s2[4,0]],\
                                               [β_s3[3,0], β_s3[4,0], β_s3[5,0]]]))
        Ass_iter += 1
        if np.max(abs(np.linalg.eigvals(Ass)))<0.99:
            Ass_stable = True
        elif Ass_iter > stability_max_iter:
            print('Ass Unstable')
        else:
            Ass_stable = False
    
    Bs = sp.linalg.solve(J_inv, np.diag([σ2_s1,σ2_s2,σ2_s3])**0.5)
    θt_next = [Azo, Azz, Bz, Aso, Ass, Bs, J_inv, λ_iter, Ass_iter]
    
    Bz1 = Bz[[0],:]
    Zt = Xt[0:2,:]; Zt1 = Zt[0,0]; Zt2 = Zt[1,0]
    St = Xt[2:5,:]
    ones = np.ones([3,1])

    Φ = sp.linalg.solve(ones@Bz1@Bz1.T@ones.T + Bs@Bs.T, ones@Bz1@Bz.T)
    Γ = sp.linalg.solve(ones@Bz1@Bz1.T@ones.T + Bs@Bs.T, Bs@Bs.T)
    
    mean = np.vstack([Azo + Azz@Zt + Φ.T@(Dt_next-ones*Zt1 - ones*Zt2 - Aso - Ass@St),\
                      Aso + Ass@St + Γ.T@(Dt_next-ones*Zt1 - ones*Zt2 - Aso - Ass@St)])
    cov = np.vstack([np.hstack([Bz@Bz.T, np.zeros([2,3])]),\
                     np.hstack([np.zeros([3,2]), Bs@Bs.T])]) -\
          np.vstack([Φ.T, Γ.T]) @ (ones@Bz1@Bz1.T@ones.T+Bs@Bs.T)@np.hstack([Φ, Γ])

    Xt_next = sp.stats.multivariate_normal.rvs(mean.flatten(), cov).reshape(-1,1)
    
    St1 = Xt[2,0]; St2 = Xt[3,0]; St3 = Xt[4,0];   
    Zt_next_1 = Xt_next[0,0];  Zt_next_2 = Xt_next[1,0];  St_next_1 = Xt_next[2,0];  St_next_2 = Xt_next[3,0];  St_next_3 = Xt_next[4,0];  
    
    first_eq_Rt_next = np.array([[Zt1],[Zt2]])
    first_eq_Zt_next = Zt_next_1
    first_eq_bt_next = np.array([[1.0],[1.0]])
    first_eq_Λt_next = Ht[0][1] + first_eq_Rt_next@first_eq_Rt_next.T
    first_eq_ct_next = Ht[0][2] + 1.0
    first_eq_dt_next = (first_eq_Zt_next - first_eq_Rt_next.T@first_eq_bt_next)**2  + Ht[0][3]
    
    second_eq_bt_next, second_eq_Λt_next, second_eq_ct_next, second_eq_dt_next = \
    bayes_para_update(Ht[1][0], Ht[1][1], Ht[1][2], Ht[1][3], np.array([[1],[Zt2]]), Zt_next_2)
    
    third_eq_bt_next, third_eq_Λt_next, third_eq_ct_next, third_eq_dt_next = \
    bayes_para_update(Ht[2][0], Ht[2][1], Ht[2][2], Ht[2][3], np.array([[St1],[St2],[St3]]), St_next_1)
    
    fourth_eq_bt_next, fourth_eq_Λt_next, fourth_eq_ct_next, fourth_eq_dt_next = \
    bayes_para_update(Ht[3][0], Ht[3][1], Ht[3][2], Ht[3][3], np.array([[1],[St_next_1],[St1],[St2],[St3]]), St_next_2)
    
    fifth_eq_bt_next, fifth_eq_Λt_next, fifth_eq_ct_next, fifth_eq_dt_next = \
    bayes_para_update(Ht[4][0], Ht[4][1], Ht[4][2], Ht[4][3], np.array([[1],[St_next_1],[St_next_2],[St1],[St2],[St3]]), St_next_3)
    
    Ht_next = [[first_eq_bt_next, first_eq_Λt_next, first_eq_ct_next, first_eq_dt_next],\
               [second_eq_bt_next, second_eq_Λt_next, second_eq_ct_next, second_eq_dt_next],\
               [third_eq_bt_next, third_eq_Λt_next, third_eq_ct_next, third_eq_dt_next],\
               [fourth_eq_bt_next, fourth_eq_Λt_next, fourth_eq_ct_next, fourth_eq_dt_next],\
               [fifth_eq_bt_next, fifth_eq_Λt_next, fifth_eq_ct_next, fifth_eq_dt_next]]

    des_mean = ones*Zt1 +ones*Zt2 + Aso + Ass@St
    des_cov = ones@Bz1@Bz1.T@ones.T+Bs@Bs.T
    
    density = sp.stats.multivariate_normal.pdf(Dt_next.flatten(), des_mean.flatten(), des_cov)
    
    return θt_next, Xt_next, Ht_next, density

In [9]:
np.random.seed(seed)
start_time = time.time()

D_0 = obs_series[:,[0]]
θ_t_particle = []
X_t_particle = []
H_t_particle = []
for i in range(N):
    θ_0, X_0, H_0, _ = init(D_0, Λ_scale, cd_scale)
    θ_t_particle.append(θ_0)
    X_t_particle.append(X_0)
    H_t_particle.append(H_0)

with open(casedir + 'θ_0.pkl', 'wb') as f:
    pickle.dump(θ_t_particle, f)
with open(casedir + 'X_0.pkl', 'wb') as f:
    pickle.dump(X_t_particle, f)
with open(casedir + 'H_0.pkl', 'wb') as f:
    pickle.dump(H_t_particle, f)
with open(casedir + 'count_0.pkl', 'wb') as f:
    pickle.dump(list(np.ones(N)), f)
with open(casedir + 'w_0.pkl', 'wb') as f:
    pickle.dump(list(np.ones(N)/N), f)
run_time = time.time() - start_time
print(run_time)    

del(θ_t_particle)

for t in tqdm(range(T-1)):
    
    D_t_next = obs_series[:,[t+1]]
    
    θ_t_next_particle = []
    X_t_next_particle = []
    H_t_next_particle = []
    ν_t_next_particle = []
    
    for i in range(N):

        θ_t_next, X_t_next, H_t_next, ν_t_next = recursive(D_t_next, X_t_particle[i], H_t_particle[i])
        θ_t_next_particle.append(θ_t_next)
        X_t_next_particle.append(X_t_next)
        H_t_next_particle.append(H_t_next)
        ν_t_next_particle.append(ν_t_next)
    
    with open(casedir + 'θ_' + str(t+1) + '.pkl', 'wb') as f:
        pickle.dump(θ_t_next_particle, f)
    with open(casedir + 'X_' + str(t+1) + '.pkl', 'wb') as f:
        pickle.dump(X_t_next_particle, f)
    with open(casedir + 'H_' + str(t+1) + '.pkl', 'wb') as f:
        pickle.dump(H_t_next_particle, f)
    
    w_t_next = ν_t_next_particle/np.sum(ν_t_next_particle)
    try:
        count_all = sp.stats.multinomial.rvs(N, w_t_next)
    except:
        for i in range(w_t_next.shape[0]):
            if w_t_next[i]>(np.sum(w_t_next[:-1]) - 1):
                w_t_next[i] = w_t_next[i] - (np.sum(w[:-1]) - 1)
                break
        count_all = sp.stats.multinomial.rvs(N, w_t_next)
    
    with open(casedir + 'w_' + str(t+1) + '.pkl', 'wb') as f:
        pickle.dump(w_t_next, f)
    with open(casedir + 'count_' + str(t+1) + '.pkl', 'wb') as f:
        pickle.dump(count_all, f)
    
    del(X_t_particle)
    del(H_t_particle)
    del(w_t_next)
    del(D_t_next)
    del(θ_t_next_particle)
    del(ν_t_next_particle)
    
    X_t_particle = []
    H_t_particle = []
    
    for i in range(N):
        if count_all[i] != 0:
            for n in range(count_all[i]):
                X_t_particle.append(X_t_next_particle[i])
                H_t_particle.append(H_t_next_particle[i])
    
    del(count_all)            
    del(X_t_next_particle)
    del(H_t_next_particle)
    

26.06269907951355


  1%|█▊                                                                                                                                                                   | 3/282 [00:33<51:43, 11.12s/it]


KeyboardInterrupt: 

In [10]:
@njit
def bayes_para_update(bt, Λt, ct, dt, Rt_next, Zt_next):
    
    Λt_next = Λt + Rt_next@Rt_next.T
    bt_next = np.linalg.solve(Λt_next, Λt@bt + Rt_next*Zt_next)
    ct_next = ct + 1.0
    dt_next = Zt_next**2 - bt_next.T@Λt_next@bt_next + bt.T@Λt@bt + dt
    
    return bt_next, Λt_next, ct_next, dt_next

In [11]:
def draw_para(b, Λ, c, d):
    
    ζ = sp.stats.gamma.rvs(c/2+1, loc = 0, scale = 1/(d/2))
    σ2 = 1/ζ
    β = np.random.multivariate_normal(b.flatten(), sp.linalg.inv(ζ*Λ)).reshape(-1,1)
    return β, σ2